### Notes for cleaning:
- False investments to clean:
    - Per http://www.sharkalytics.com/company/classroom-jams - Mark Cuban was not present, caught 50000 investment under Mark name that is not true. Need to remove. - COMPLETE
    - Per http://www.sharkalytics.com/company/the-beebo - Barbara investment was actually Lori, Barabara wasn't present, pulled up show to confirm. - COMPLETE
    - Per http://www.sharkalytics.com/company/shark-wheel - Robert investment was actually Kevin. - COMPLETE
    - Per http://www.sharkalytics.com/company/pursecase - Daymon was not present, Lori actually made this deal. - COMPLETE

- Analysis of main sharks only, guest data/columns dropped.

In [1]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
from src.util import check_nulls


In [2]:
df = pd.read_csv('datasets/Shark Tank US dataset.csv')

In [3]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [4]:
df.columns

Index(['season_number', 'startup_name', 'episode_number', 'pitch_number',
       'season_start', 'season_end', 'original_air_date', 'industry',
       'business_description', 'company_website', 'pitchers_gender',
       'pitchers_average_age', 'pitchers_city', 'pitchers_state',
       'entrepreneur_names', 'multiple_entrepreneurs', 'us_viewership',
       'original_ask_amount', 'original_offered_equity', 'valuation_requested',
       'got_deal', 'total_deal_amount', 'total_deal_equity', 'deal_valuation',
       'number_of_sharks_in_deal', 'investment_amount_per_shark',
       'equity_per_shark', 'royalty_deal', 'advisory_shares_equity', 'loan',
       'deal_has_conditions', 'barbara_corcoran_investment_amount',
       'barbara_corcoran_investment_equity', 'mark_cuban_investment_amount',
       'mark_cuban_investment_equity', 'lori_greiner_investment_amount',
       'lori_greiner_investment_equity', 'robert_herjavec_investment_amount',
       'robert_herjavec_investment_equity', 'daymon

In [5]:
check_nulls(df)

{'season_number': 0,
 'startup_name': {'standard_nulls': 0,
  'empty_strings': 0,
  'specific_values': 0,
  'whitespace_only': 0},
 'episode_number': 0,
 'pitch_number': 0,
 'season_start': {'standard_nulls': 0,
  'empty_strings': 0,
  'specific_values': 0,
  'whitespace_only': 0},
 'season_end': {'standard_nulls': 0,
  'empty_strings': 0,
  'specific_values': 0,
  'whitespace_only': 0},
 'original_air_date': {'standard_nulls': 0,
  'empty_strings': 0,
  'specific_values': 0,
  'whitespace_only': 0},
 'industry': {'standard_nulls': 0,
  'empty_strings': 0,
  'specific_values': 0,
  'whitespace_only': 0},
 'business_description': {'standard_nulls': 0,
  'empty_strings': 0,
  'specific_values': 0,
  'whitespace_only': 0},
 'company_website': {'standard_nulls': 758,
  'empty_strings': 0,
  'specific_values': 0,
  'whitespace_only': 0},
 'pitchers_gender': {'standard_nulls': 7,
  'empty_strings': 0,
  'specific_values': 0,
  'whitespace_only': 0},
 'pitchers_average_age': {'standard_nulls'

In [6]:
sharks = [x for x in df.columns if 'present' in x]
investment_columns = [x for x in df.columns if 'investment_amount' in x]

investment_columns.remove('investment_amount_per_shark')
investment_columns.remove('guest_investment_amount')
sharks.remove('guest_present')

shark_investments = list(zip(sharks,investment_columns))
shark_investments

[('barbara_corcoran_present', 'barbara_corcoran_investment_amount'),
 ('mark_cuban_present', 'mark_cuban_investment_amount'),
 ('lori_greiner_present', 'lori_greiner_investment_amount'),
 ('robert_herjavec_present', 'robert_herjavec_investment_amount'),
 ('daymond_john_present', 'daymond_john_investment_amount'),
 ('kevin_o_leary_present', 'kevin_o_leary_investment_amount')]

In [7]:
# Before Fix
for x in shark_investments:
    print(df.loc[df['got_deal'] == True].groupby(x[0])[x[1]].mean().values[0], x[0])
    print(df.loc[df['got_deal'] == True].groupby(x[0])[x[1]].sum().values[0], x[1])

100000.0 barbara_corcoran_present
100000.0 barbara_corcoran_investment_amount
50000.0 mark_cuban_present
50000.0 mark_cuban_investment_amount
nan lori_greiner_present
0.0 lori_greiner_investment_amount
75000.0 robert_herjavec_present
75000.0 robert_herjavec_investment_amount
55000.0 daymond_john_present
55000.0 daymond_john_investment_amount
nan kevin_o_leary_present
0.0 kevin_o_leary_investment_amount


In [8]:
# REMOVED ROW 6 MARK CUBAN FALSE INVESTMENT of 50000
df.loc[(df['got_deal'] == True)&(df['mark_cuban_investment_amount'] == 50000)&(df['mark_cuban_present'] == False)][investment_columns]
df.loc[6, 'mark_cuban_investment_amount'] = 0

In [9]:
# MOVED ROW 495 BARBARA INVESTMENT TO LORI
df.loc[(df['got_deal'] == True)&(df['barbara_corcoran_investment_amount'] == 100000)&(df['barbara_corcoran_present'] == False)][investment_columns]
df.loc[495, 'barbara_corcoran_investment_amount'] = 0
df.loc[495, 'lori_greiner_investment_amount'] = 100000
df.loc[495, 'barbara_corcoran_investment_equity'] = 0
df.loc[495, 'lori_greiner_investment_equity'] = 15.0


In [10]:
# MOVED ROW 491 ROBERT INVESTMENT TO KEVIN
df.loc[(df['got_deal'] == True)&(df['robert_herjavec_investment_amount'] == 75000)&(df['robert_herjavec_present'] == False)][investment_columns]
df.loc[491, 'robert_herjavec_investment_amount'] = 0
df.loc[491, 'kevin_o_leary_investment_amount'] = 75000
df.loc[491, 'robert_herjavec_investment_equity'] = 0
df.loc[491, 'kevin_o_leary_investment_equity'] = 2.5



In [11]:
# MOVED ROW 301 DAYMOND INVESTMENT TO LORI
df.loc[(df['got_deal'] == True)&(df['daymond_john_investment_amount'] == 55000)&(df['daymond_john_present'] == False)][investment_columns]
df.loc[301, 'daymond_john_investment_amount'] = 0
df.loc[301, 'lori_greiner_investment_amount'] = 55000
df.loc[301, 'daymond_john_investment_equity'] = 0
df.loc[301, 'lori_greiner_investment_equity'] = 15.0




In [12]:
# AFTER FIX
for x in shark_investments:
    print(df.loc[df['got_deal'] == True].groupby(x[0])[x[1]].mean().values[0], x[0])
    print(df.loc[df['got_deal'] == True].groupby(x[0])[x[1]].sum().values[0], x[1])
    

0.0 barbara_corcoran_present
0.0 barbara_corcoran_investment_amount
0.0 mark_cuban_present
0.0 mark_cuban_investment_amount
nan lori_greiner_present
0.0 lori_greiner_investment_amount
0.0 robert_herjavec_present
0.0 robert_herjavec_investment_amount
0.0 daymond_john_present
0.0 daymond_john_investment_amount
nan kevin_o_leary_present
0.0 kevin_o_leary_investment_amount


In [13]:
check_nulls(df)

{'season_number': 0,
 'startup_name': {'standard_nulls': 0,
  'empty_strings': 0,
  'specific_values': 0,
  'whitespace_only': 0},
 'episode_number': 0,
 'pitch_number': 0,
 'season_start': {'standard_nulls': 0,
  'empty_strings': 0,
  'specific_values': 0,
  'whitespace_only': 0},
 'season_end': {'standard_nulls': 0,
  'empty_strings': 0,
  'specific_values': 0,
  'whitespace_only': 0},
 'original_air_date': {'standard_nulls': 0,
  'empty_strings': 0,
  'specific_values': 0,
  'whitespace_only': 0},
 'industry': {'standard_nulls': 0,
  'empty_strings': 0,
  'specific_values': 0,
  'whitespace_only': 0},
 'business_description': {'standard_nulls': 0,
  'empty_strings': 0,
  'specific_values': 0,
  'whitespace_only': 0},
 'company_website': {'standard_nulls': 758,
  'empty_strings': 0,
  'specific_values': 0,
  'whitespace_only': 0},
 'pitchers_gender': {'standard_nulls': 7,
  'empty_strings': 0,
  'specific_values': 0,
  'whitespace_only': 0},
 'pitchers_average_age': {'standard_nulls'

In [14]:
guest_columns = [x for x in df.columns if 'guest' in x]
guest_columns

['guest_investment_amount',
 'guest_investment_equity',
 'guest_name',
 'guest_present']

In [15]:
df.drop(guest_columns, axis=1, inplace=True)

In [17]:
guest_columns = [x for x in df.columns if 'guest' in x]
guest_columns

[]